# Base64 エンコーディング
https://algoful.com/Archive/Algorithm/Base64

エンコード方式の 1 つで、64 種類の印字可能な英数字記号を用いてデータをエンコードします。  
広く使われるエンコード方式であり、以下の様な利用実績がある。

- 電子メールのマルチバイト文字のエンコード
- 画像ファイルのエンコード
- Basic 認証  
  などなど

## 特徴

Base64 は、データを 64 種類の文字(A–Z, a–z, 0–9, +, /)とパディング用の文字(=)にエンコードします。  
後述の通り、6bit ずつに分解して対応する 64 種の文字に置き換えていくので、データサイズにかかわらずエンコード後のデータサイズは約 33%(6bit->8bit なので)増加します。

約 33%となっているのはパディング用の文字(=)が追加される場合があるため。  
ただし基本 3 割増しのデータ量になることは覚えておこう！

### e.g.

`1位`という UTF8 の文字を Base64 でエンコードすると`MeS9jQ==`となる。  
半角文字の "1" も、全角文字の "位" も別の半角英数に置き換えられている。


# Base64 のアルゴリズム詳細

## エンコード

1. エンコード対象のデータを 6bit ずつに分解します。データが 6bit に足りない分は 0 で埋めます。
   - 例: `01100001` -> `011000` + `010000`
2. 分解して得られた各 6bit のデータを、4 文字ずつ対応表を使って文字に置き換えます。4 文字に満たない分は "=" で埋めます。
   - 例: `011000` + `010000` -> `YQ==`  
     つまり、エンコード後の Base64 の文字数は必ず 4 倍数になるということ！

6bit の対応表は、以下サイトを参考にしよう  
[Base64(wikipedia)](https://ja.wikipedia.org/wiki/Base64)

## デコード

デコードはエンコードの手順を逆から実行していくだけ。

1. 対応表から bit データに置き換える。_この時、"=" は無視する_。
   - 例: `YQ==` -> `011000` + `010000`
2. 8bit に満たない末尾の 0 は切り捨てます。
   - 例: `011000` + `010000` -> `01100001`


In [19]:
# pythonでの実装例


class Base64:

    def __init__(self) -> None:
        self.base64table = {
            "000000": "A",
            "000001": "B",
            "000010": "C",
            "000011": "D",
            "000100": "E",
            "000101": "F",
            "000110": "G",
            "000111": "H",
            "001000": "I",
            "001001": "J",
            "001010": "K",
            "001011": "L",
            "001100": "M",
            "001101": "N",
            "001110": "O",
            "001111": "P",
            "010000": "Q",
            "010001": "R",
            "010010": "S",
            "010011": "T",
            "010100": "U",
            "010101": "V",
            "010110": "W",
            "010111": "X",
            "011000": "Y",
            "011001": "Z",
            "011010": "a",
            "011011": "b",
            "011100": "c",
            "011101": "d",
            "011110": "e",
            "011111": "f",
            "100000": "g",
            "100001": "h",
            "100010": "i",
            "100011": "j",
            "100100": "k",
            "100101": "l",
            "100110": "m",
            "100111": "n",
            "101000": "o",
            "101001": "p",
            "101010": "q",
            "101011": "r",
            "101100": "s",
            "101101": "t",
            "101110": "u",
            "101111": "v",
            "110000": "w",
            "110001": "x",
            "110010": "y",
            "110011": "z",
            "110100": "0",
            "110101": "1",
            "110110": "2",
            "110111": "3",
            "111000": "4",
            "111001": "5",
            "111010": "6",
            "111011": "7",
            "111100": "8",
            "111101": "9",
            "111110": "+",
            "111111": "/",
        }

    def encode(self, plane_text: str):
        result = []

        #文字列をbyte型配列に変換 (byteなので、8bitの範囲で表現できる-128 ~ 127までの値を取ることができる)
        byte = plane_text.encode("utf-8")
        # さらにbinaryに変換
        binary = []
        for x in byte:
            # 長さ8のbinary表示に変換
            binary.append(f'{x:08b}')
        binary = "".join(binary)

        bit6 = ""
        for bit in binary:
            bit6 += bit

            if len(bit6) == 6:
                result.append(self.base64table[bit6])
                bit6 = ""
        
        if len(bit6) != 0:
            padCount = 6-len(bit6)
            bit6 += "0"*padCount
            result.append(self.base64table[bit6])

        # 出力文字数が4の倍数になるように "=" を付け足す
        while len(result) % 4 != 0:
            result.append("=")

        return "".join(result)


    def decode(self, code: str):
        table_keys = list(self.base64table.keys())
        table_values = list(self.base64table.values())

        bit8 = ""
        # byte = bytes()
        byte = ""
        result = ""

        for char in code:
            if char != "=":
                key_pos = table_values.index(char)
                bit6 = table_keys[key_pos]

                for bit in bit6:
                    bit8 += bit

                    if len(bit8) == 8:
                        byte += bit8
                        result += chr(int(bit8, 2))
                        bit8 = ""


        return result
        # return "".join([chr(int(x,2)) for x in [byte[i:i+8] for i in range(0,len(byte), 8)]])






encoder = Base64()
code = encoder.encode("HFAwsefaHUARff")
print(code)
decoded = encoder.decode(code)
print(decoded)


SEZBd3NlZmFIVUFSZmY=
HFAwsefaHUARff


### python での byte, binary の扱い

binary とは、0,1 で表現されたもの（Bit）ですが、実際には、python は Byte の扱いしかサポートしてない。  
=> encode メソッドで byte 型に変換した後、teratail にあるように for 文を用いて一文字ずつ binary に変換していく必要がある

以下の 2 記事を参考。  
https://qiita.com/Gyutan/items/7939a723c117c8ebbe5f  
https://teratail.com/questions/192403


## 結果の確認
以下のサイトでブラウザ上でbase64エンコーディングをお試しできる。  
実装したコードと同じ結果を返すことが確認できた。  
https://www.en-pc.jp/tech/base64.php#result